# 11.  최적 군집화 중 가중치를 부여하여 가장 취약한 10개 지역 선정

In [ ]:
import pandas as pd
from pandas import DataFrame
import numbers
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
raw_df=pd.read_csv("grade2.csv", encoding='euc-kr') # 선택한 군집 파일

In [ ]:
raw_df.head(3)

### minmaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
cluster_df=raw_df.drop(['name', 'Unnamed: 0', 'cluster_grade', 'X', 'Y'], inplace=False, axis=1)

In [ ]:
cluster_df.head(3)

In [ ]:
# 변수의 서로 다른 값을 표준화하기 위해 minmax 실행
scaler=MinMaxScaler()
scaler.fit(cluster_df)
df_scaled=scaler.transform(cluster_df)

In [ ]:
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=cluster_df.columns

In [ ]:
df_scaled

In [ ]:
# 상관계수를 토대로 가중치 부여

In [ ]:
# 가중치계산을 위해 0 -> 0.000001
df=df_scaled.replace(0, 0.000001)

In [ ]:
# 상관계수를 토대로 가중치 부여

In [ ]:
# 상관계수
welfare=0.98; library=0.96; theater=0.84;
# 문화서비스는 하나의 영역으로 처리
service=(welfare+library+theater)/3
busStop=0.79; hospital=0.82; store=0.91
busTime=0.51; age=0.97

In [ ]:
# 가중치 비율
total=service+busStop+hospital+store+busTime+age

service=service/total
busStop=busStop/total
hospital=hospital/total
store=store/total
busTime=busTime/total
age=age/total

service=service/3

In [ ]:
print(service, busStop, hospital, store, busTime, age)

In [ ]:
# 가중치 dataframe
weight=pd.Series({"busStop":busStop, "hospital":hospital, "store":store, "welfare":service, "library":service, "theater":service, "busTime":busTime, "age":age})

In [ ]:
weight=pd.DataFrame(weight)
weight.columns=['weight']

In [ ]:
weight

In [ ]:
# 가중치 곱한 값
a=df['busStop_distance']*busStop
b=df['hospital_distance']*hospital
c=df['store_distance']*store
d=df['welfare_distance']*welfare
e=df['library_distance']*library
f=df['theater_distance']*theater
g=df['busTime_mean']*busTime
h=df['aged_population_ratio']*age

In [ ]:
# 가중치 곱한 최종
final=pd.concat([a,b,c,d,e,f,g,h], axis=1, sort=True)
final

In [ ]:
#평균값 구하기
mean_df=final.mean(axis=1)

mean_df=pd.DataFrame(mean_df)
mean_df.columns=['mean']
mean_df

In [ ]:
rank_df=pd.concat([raw_df['name'],mean_df], axis=1, join='inner')

In [ ]:
# 내림차순 정렬
rank=rank_df.sort_values(by=['mean'], axis=0, ascending=False)
#rank

In [ ]:
# rank 10 추출
rank=rank.drop_duplicates(['mean'])
rank_top10=rank.iloc[0:10, :]
rank_top10

In [ ]:
senior_df=pd.read_csv("final_senior_center.csv", encoding='utf-8')
jeonju_df=senior_df["borough"].str.contains("전주시")
senior_center=senior_df[jeonju_df] # 전주시 마을회관 데이터
senior_center.head(2)

In [ ]:
senior_center=senior_center[['name', 'X', 'Y']] # 필요 column 추출
#senior_center

In [ ]:
# rank top 10 의 X Y 좌표 연결
final_rank_10=pd.merge(rank_top10, senior_center, on='name', how='left')
final_rank_10=final_rank_10.drop([final_rank_10.index[7]]) # 잘못된 값 삭제
final_rank_10

In [ ]:
final_rank_10.to_csv("final_rank_10.csv", encoding='euc-kr') # csv 저장